In [5]:
import pandas as pd
from plots.utils import compute_user_penalty, get_team_values_df
from common.load import load_competition_data, process_team_logs

# Import common data

In [6]:
config = 'config_vbs2023.yaml'

# load competition data from dres files and auxiliary data (FPSs, sequences)
comp_data = load_competition_data(config)

# compute team logs and put them in the form of dataframes (or load them if already existing)
logs = process_team_logs(config, comp_data, force=False)

teams = comp_data['config']['teams']

/disks/workspace/Workspace/VBS-Analysis/VBS-2022/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/VBS-Analysis/VBS-2022/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/VBS-Analysis/VBS-2022/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/VBS-Analysis/VBS-2022/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/V

# Analysis

## Time Recall Table

In [7]:
def compute_data_for_time_recall_table(comp_data, teams, logs, split_user=False, only_best_user=False, max_records=10000):
    dfs = []
    for team in teams:
        team_df = logs[team].get_events_dataframe().reset_index()
        team_df = get_team_values_df(comp_data, team_df, split_user if not only_best_user else True, max_records)
        dfs.append(team_df)

    total_df = pd.concat(dfs, axis=0).reset_index()

    # keep the results from the best user among the two, for each given (task, team)
    if only_best_user:
        # the best penalty contribution is given by the ranks. If they are equal, then the user that submitted earlier wins
        user_penalty = compute_user_penalty(total_df, max_records)
                    
        total_df['user_penalty'] = user_penalty
        total_df = total_df.loc[total_df.groupby(['team', 'task'])['user_penalty'].idxmin()]
        total_df = total_df.drop(['user_penalty'], axis=1)
    
    return total_df

def build_table(df, teams):
    # drop unuseful columns
    df = df.drop(['time_first_appearance', 'rank_shot_first_appearance', 'time_last_appearance', 'rank_shot_last_appearance', 'time_first_appearance_video', 'rank_video_first_appearance'], axis=1)

    #renaming task
    rename_fun = lambda x: x.replace('vbs22-kis-t', 'T_').replace('vbs22-kis-v', 'V_')
    df['task'] = df['task'].apply(rename_fun)
    tasks = df[['task','task_start']].sort_values(by=['task_start'])['task'].unique()
    textual=[t for t in tasks if t.startswith('T')] #used later to order the column
    visual=[t for t in tasks if t.startswith('V')] #used later to order the column
    df.drop(columns='task_start', inplace=True)

    df = df.fillna(-1)
    col=[c for c in df.columns.values.tolist() if c!='team' and c!='task' and c!='user']
    df[col]= df[col].astype('int32')
    df[col] = df[col].applymap(lambda x: -1 if x< 0 else x)
    df = df.astype('str')
    df.replace(['-1'], '-', inplace=True)

    #aggregate
    agg_dic={c: (lambda x: ' / '.join(x)) for c in col}
    agg_dic['time_correct_submission']="min"
    df=df.groupby(['team','task'])[col].agg(agg_dic).reset_index()
    df.replace('- / -', '-', regex=True, inplace=True)
    add_second= lambda x: x if x=='-' else x + 's'
    df['time_correct_submission']=df['time_correct_submission'].apply(add_second)
    df['time_best_shot']=df['time_best_shot'].apply(add_second)
    df['rank_shot_margin_5'] = df['rank_shot_margin_5'].apply(add_second)
    df['time_best_video'] = df['time_best_video'].apply(add_second)

    df = df.melt(var_name="metric", id_vars=["team", "task"], value_name="value")
    df['unit'] = df['metric'].apply(lambda x: 'rank' if x.startswith('rank_') else 'time')
    replace_dic = {
        'rank_shot_margin_0': 'correct frame',
        'time_best_shot': 'correct frame',
        'rank_shot_margin_5': 'frame in GT+2x5s',
        'time_best_shot_margin5': 'frame in GT+2x5s',
        'rank_video': 'correct video',
        'time_best_video': 'correct video',
        'time_correct_submission': 'correct submission'
    }
    df['metric']=df['metric'].map(replace_dic)
    df = df.pivot(index=['team', 'metric', 'unit'], columns="task", values="value")
    df=df.fillna('-')


    #sorting index desired order
    level_0=teams #order in the conf file
    level_1=['correct frame', 'frame in GT+2x5s', 'correct video','correct submission']
    level_2=['rank','time']
    df = df.reindex(pd.MultiIndex.from_product([level_0,level_1,level_2]))
    df.dropna(axis=0, inplace=True)#'correct submission'/rank shluld not be in the index
    df_margin_5 = df.copy()

    # sorting index desired order
    level_0 = teams  # order in teh conf file
    level_1 = ['correct frame', 'correct video', 'correct submission']
    level_2 = ['rank', 'time']
    df = df.reindex(pd.MultiIndex.from_product([level_0, level_1, level_2]))
    df.dropna(axis=0, inplace=True)  # 'correct submission'/rank shluld not be in the index

    #ordering columns
    df=df[textual+visual]

    return df, df_margin_5

In [8]:
df = compute_data_for_time_recall_table(comp_data, teams, logs)
table, table5 = build_table(df, teams)
table5

task                               vbs23-kis-t1 vbs23-kis-t2 vbs23-kis-t3  \
VISIONE    correct frame      rank            1            1            1   
                              time          26s          65s         384s   
           frame in GT+2x5s   rank           1s           1s           1s   
                              time           26           65          384   
           correct video      rank            1            1            1   
                              time          26s          59s         384s   
           correct submission time          65s          76s            -   
VideoCLIP  correct frame      rank            1         1335            -   
                              time          51s         334s            -   
           frame in GT+2x5s   rank           1s        1335s          13s   
                              time           51          334          191   
           correct video      rank            1         1335           13   
                              time          51s         334s         191s   
           correct submission time         283s            -            -   
Verge      correct frame      rank            3           79          638   
                              time          40s         160s         289s   
           frame in GT+2x5s   rank           3s          79s         180s   
                              time           40          160          382   
           correct video      rank            3           79          124   
                              time          40s         160s         210s   
           correct submission time         281s            -            -   
HTW        correct frame      rank            1            1           12   
                              time          28s           6s         277s   
           frame in GT+2x5s   rank           1s           1s          12s   
                              time           28            6          277   
           correct video      rank            1            1           12   
                              time          28s           6s         277s   
           correct submission time          37s          26s         395s   
VIREO      correct frame      rank            6          141          100   
                              time         137s         409s          53s   
           frame in GT+2x5s   rank           6s         141s          92s   
                              time          137          409           38   
           correct video      rank            1          141           64   
                              time          20s         409s         280s   
           correct submission time         166s            -            -   
vitrivr    correct frame      rank            4            4            2   
                              time          23s          78s          32s   
           frame in GT+2x5s   rank           4s           4s           2s   
                              time           23           78           32   
           correct video      rank            2            4            2   
                              time          47s          78s          32s   
           correct submission time          94s         219s            -   
vitrivr-VR correct frame      rank            -            -            -   
                              time         124s         153s          40s   
           frame in GT+2x5s   rank            -            -            -   
                              time          124          153           40   
           correct video      rank            -            -            3   
                              time         124s         153s          40s   
           correct submission time         213s            -          93s   

task                               vbs23-kis-t4 vbs23-kis-t5 vbs23-kis-t6  \
VISIONE    correct frame      rank           16            1            